In [1]:
# use os functions to set path
# rename site variables to mimic sample data?

import os
import pandas as pd
import numpy as np
import timeit
import time
import cudf as cf
import dask_cudf as dc
import tracemalloc

date_time = time.strftime('%Y%m%d-%H%M')


In [2]:
%timeit sites = pd.read_csv('ml_cm_sites_2020_01.csv', delimiter=',')
%timeit hr_sites = pd.read_csv('ml_cm_hourly_sites_2020_01.csv', delimiter=',')

%timeit sites = cf.read_csv('ml_cm_sites_2020_01.csv', delimiter=',')
%timeit hr_sites = cf.read_csv('ml_cm_hourly_sites_2020_01.csv', delimiter=',')

# sites = pd.read_csv('sites.csv', delimiter=',')
# hr_sites = pd.read_csv('hourly_sites.csv', delimiter=',')

# sites = cf.read_csv('sites.csv', delimiter=',')
# hr_sites = cf.read_csv('hourly_sites.csv', delimiter=',')

sites = cf.read_csv('ml_cm_sites_2020_01.csv', delimiter=',')
hr_sites = cf.read_csv('hourly_sites.csv', delimiter=',')

npv_values = pd.read_csv('npv.csv', delimiter=',')
lte_params = pd.read_csv('lte_assumptions.csv', delimiter=',')

# init_sites = sites.drop(['Type', 'Morphology'], axis=1)
# init_hr_sites = hr_sites

npv_values.columns = ['non', 'off_du', 'off_u', 'off_s', 'on_du', 'on_u',
                      'on_s', 'roe_du', 'roe_u', 'roe_s']

st_date = pd.date_range(start=pd.Timestamp('2019-01-01'), periods=11, freq='AS')

disc_rt = .15
other = 60.0
sites['rx_sensitivity_db'] = -93.3539524119117




2.33 s ± 4.24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.7 s ± 9.28 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
378 ms ± 6.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
546 ms ± 4.42 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
# functions


def morph(type, morph):
    if type == 'Off-Strand' and morph == 'Dense Urban':
        morph_code = 1
    elif type == 'Off-Strand' and morph == 'Urban':
        morph_code = 2
    elif type == 'Off-Strand' and morph == 'Suburban':
        morph_code = 3
    elif type == 'On-Strand' and morph == 'Dense Urban':
        morph_code = 4
    elif type == 'On-Strand' and morph == 'Urban':
        morph_code = 5
    elif type == 'On-Strand' and morph == 'Suburban':
        morph_code = 6
    elif type == 'ROE Building' and morph == 'Dense Urban':
        morph_code = 7
    elif type == 'ROE Building' and morph == 'Urban':
        morph_code = 8
    elif type == 'ROE Building' and morph == 'Suburban':
        morph_code = 9
    else:
        print('error')
    return morph_code


def fin_arrays(code):
    cpx = np.empty([11, 1])
    opx = np.empty([11, 1])
    growth = np.empty([11, 1])
    mvno = np.empty([11, 1])
    if code == 1:
        npv = npv_values.loc[:, 'off_du']
    elif code == 2:
        npv = npv_values.loc[:, 'off_u']
    elif code == 3:
        npv = npv_values.loc[:, 'off_s']
    elif code == 4:
        npv = npv_values.loc[:, 'on_du']
    elif code == 5:
        npv = npv_values.loc[:, 'on_u']
    elif code == 6:
        npv = npv_values.loc[:, 'on_s']
    elif code == 7:
        npv = npv_values.loc[:, 'roe_du']
    elif code == 8:
        npv = npv_values.loc[:, 'roe_u']
    elif code == 9:
        npv = npv_values.loc[:, 'roe_s']
    else:
        print('error')
    penet_rate = np.full((11, 1), float(npv[62]))
    for i in range(11):
        cpx[i] = float(npv[i + 3])
    for i in range(11):
        opx[i] = float(npv[i + 14])
    for i in range(11):
        growth[i] = float(npv[i + 40])
    for i in range(11):
        mvno[i] = float(npv[i + 51])
    array = np.hstack((cpx, opx, growth, mvno, penet_rate))
    return array


def morph_array(code):
    if code == 1:
        return off_du
    elif code == 2:
        return off_u
    elif code == 3:
        return off_s
    elif code == 4:
        return on_du
    elif code == 5:
        return on_u
    elif code == 6:
        return on_s
    elif code == 7:
        return roe_du
    elif code == 8:
        return roe_u
    elif code == 9:
        return roe_s
    else:
        print('error')


day_diff = 0.0
cell_split = 2.0


def xnpv(gbs, mo_cap, code):
    # slice 0 is cpx, 1 is opx, 2 is growth, 3 is mvno, 4 is penet_rate
    array = morph_array(code)
    flag = 0
    a = 0
    for i in range(11):
        if (gbs * array[i][4] * (array[i][2]) * 12) < (
                cell_split * mo_cap * 12):
            if (gbs * array[i][4] * (array[i][2]) * 12) < (mo_cap * 12):
                value[i] = gbs * array[i][4] * 12 * array[i][3] * array[i][2] - array[i][1] - other
            else:
                value[i] = gbs * array[i][4] * 12 * array[i][3] * array[i][2] - (2 * array[i][1]) - other
                if flag == 0:
                    a = i
                    flag = 1
        else:
            value[i] = gbs * array[i][4] * 12 * array[i][3] * array[i][2] - (2 * array[i][1]) - other
    date_0 = st_date[0]
    value[a] = value[a] - array[a][0]
    return int(sum([value_i / ((1.0 + disc_rt) **
                               (((date_i - date_0).days - day_diff) / 365.0))
                    for value_i, date_i in zip(value, st_date)]))



In [4]:
off_du = fin_arrays(1)
off_u = fin_arrays(2)
off_s = fin_arrays(3)
on_du = fin_arrays(4)
on_u = fin_arrays(5)
on_s = fin_arrays(6)
roe_du = fin_arrays(7)
roe_u = fin_arrays(8)
roe_s = fin_arrays(9)
value = np.empty([11, 1])

# sites['rx_signal_strength_db'] = 37.0 - sites['path_loss_umi_db']
sites['rx_signal_strength_db'] = 37.0 - sites['path_loss_db']
sites['sinr'] = sites['rx_signal_strength_db'] - sites['rx_sensitivity_db']
sites['sinr'] = sites['sinr'].round(0)

# sites['rx_signal_strength_mw'] = ((10 ** ((37.0 -
#                                            sites['path_loss_umi_db'])
#                                           / 10) / 1000.000000000))
# bin_sinr = sites[['GridName', 'sinr', 'rx_signal_strength_mw',
#                   'rx_signal_strength_db', 'rx_sensitivity_db']]


sites['rx_signal_strength_mw'] = ((10 ** ((37.0 -
                                           sites['path_loss_db'])
                                          / 10) / 1000.000000000))
# bin_sinr = sites[['GridName', 'sinr', 'rx_signal_strength_mw',
#                   'rx_signal_strength_db', 'rx_sensitivity_db']]

sites = sites[sites.sinr >= -7.0]

In [5]:
# %timeit 

%timeit test1 = sites.groupby('asset_id').bin_usage_gb.sum().to_pandas()
test1 = sites.groupby('asset_id').bin_usage_gb.sum().to_pandas()
%timeit test2 = test1.to_frame()
test2 = test1.to_frame()
%timeit test2['asset_id'] = test1.index
test2['asset_id'] = test1.index
%timeit test1 = cf.from_pandas(test2)
test3 = cf.from_pandas(test2)
%timeit test_merge1 = sites.merge(test3, on=['asset_id'], how='left')

sites = sites.to_pandas()

%timeit sites['gb_offload'] = sites.groupby('asset_id')['bin_usage_gb'].transform(sum)



11.2 ms ± 36.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
198 µs ± 3.74 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
122 µs ± 644 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
1.42 ms ± 2.23 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
54.9 ms ± 1.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
57.2 ms ± 13.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
